In [27]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

In [28]:
dataDir = '../data/all/'

# application data

In [29]:
application_train = pd.read_csv( dataDir + 'application_train.csv' )

In [30]:
application_train.head(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# look at ddata types of each colum
print application_train.dtypes

SK_ID_CURR                        int64
TARGET                            int64
NAME_CONTRACT_TYPE               object
CODE_GENDER                      object
FLAG_OWN_CAR                     object
FLAG_OWN_REALTY                  object
CNT_CHILDREN                      int64
AMT_INCOME_TOTAL                float64
AMT_CREDIT                      float64
AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
NAME_TYPE_SUITE                  object
NAME_INCOME_TYPE                 object
NAME_EDUCATION_TYPE              object
NAME_FAMILY_STATUS               object
NAME_HOUSING_TYPE                object
REGION_POPULATION_RELATIVE      float64
DAYS_BIRTH                        int64
DAYS_EMPLOYED                     int64
DAYS_REGISTRATION               float64
DAYS_ID_PUBLISH                   int64
OWN_CAR_AGE                     float64
FLAG_MOBIL                        int64
FLAG_EMP_PHONE                    int64
FLAG_WORK_PHONE                   int64


In [32]:
#list( application_train.columns )
y = application_train['TARGET']
X = application_train.drop('TARGET', axis = 1)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42 )

In [33]:
# one hot encoding for categorical variables
one_hot_encoded_X_train = pd.get_dummies(X_train)
one_hot_encoded_X_test = pd.get_dummies(X_test)
encoded_X_train, encoded_X_test = one_hot_encoded_X_train.align(one_hot_encoded_X_test,
                                                                    join='left', 
                                                                    axis=1)

In [34]:
# check the columns after one hot encoding
print encoded_X_train.columns

Index([u'SK_ID_CURR', u'CNT_CHILDREN', u'AMT_INCOME_TOTAL', u'AMT_CREDIT',
       u'AMT_ANNUITY', u'AMT_GOODS_PRICE', u'REGION_POPULATION_RELATIVE',
       u'DAYS_BIRTH', u'DAYS_EMPLOYED', u'DAYS_REGISTRATION',
       ...
       u'HOUSETYPE_MODE_terraced house', u'WALLSMATERIAL_MODE_Block',
       u'WALLSMATERIAL_MODE_Mixed', u'WALLSMATERIAL_MODE_Monolithic',
       u'WALLSMATERIAL_MODE_Others', u'WALLSMATERIAL_MODE_Panel',
       u'WALLSMATERIAL_MODE_Stone, brick', u'WALLSMATERIAL_MODE_Wooden',
       u'EMERGENCYSTATE_MODE_No', u'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=245)


In [35]:
print X_train.shape
print encoded_X_train.shape

(276759, 121)
(276759, 245)


In [ ]:
# handle missiong values
columns = X_train.columns
missing_val_count_by_column = (X_train.isnull().sum())
print missing_val_count_by_column / float(X_train.shape[0])
# check if there is any missing y
print y_train.isnull().sum(), y_test.isnull().sum()

SK_ID_CURR                      0.000000
NAME_CONTRACT_TYPE              0.000000
CODE_GENDER                     0.000000
FLAG_OWN_CAR                    0.000000
FLAG_OWN_REALTY                 0.000000
CNT_CHILDREN                    0.000000
AMT_INCOME_TOTAL                0.000000
AMT_CREDIT                      0.000000
AMT_ANNUITY                     0.000036
AMT_GOODS_PRICE                 0.000896
NAME_TYPE_SUITE                 0.004220
NAME_INCOME_TYPE                0.000000
NAME_EDUCATION_TYPE             0.000000
NAME_FAMILY_STATUS              0.000000
NAME_HOUSING_TYPE               0.000000
REGION_POPULATION_RELATIVE      0.000000
DAYS_BIRTH                      0.000000
DAYS_EMPLOYED                   0.000000
DAYS_REGISTRATION               0.000000
DAYS_ID_PUBLISH                 0.000000
OWN_CAR_AGE                     0.659736
FLAG_MOBIL                      0.000000
FLAG_EMP_PHONE                  0.000000
FLAG_WORK_PHONE                 0.000000
FLAG_CONT_MOBILE

In [ ]:
# may need to handle each column with different strategy, but now I just impute using most frequent one
from sklearn.preprocessing import Imputer
my_imputer = Imputer(strategy = 'most_frequent')
encoded_X_train_imputed = my_imputer.fit_transform(encoded_X_train)
encoded_X_test_imputed = my_imputer.transform(encoded_X_test)


In [ ]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)
predictions = gbm.predict(X_test)

# bureau_balance

In [ ]:
application_train = pd.read_csv( dataDir + 'application_train.csv' )